In [23]:
import sys
import stlearn as st
st.settings.set_figure_params(dpi=300)
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sys
import os
file = Path("../stimage").resolve()
parent= file.parent
sys.path.append(str(parent))
from PIL import Image
from stimage._utils import gene_plot, ReadOldST, scale_img, calculate_bg, tiling
from stimage._model import CNN_NB_trainable
from stimage._data_generator import DataGenerator
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from anndata import read_h5ad
from stimage._img_normaliser import IterativeNormaliser

In [24]:
from pathlib import Path
from typing import Optional, Union
from anndata import AnnData
import pandas as pd
import stlearn
import scanpy

def ReadOldST(
        count_matrix_file: Union[str, Path] = None,
        spatial_file: Union[str, Path] = None,
        image_file: Union[str, Path] = None,
        library_id: str = "OldST",
        scale: float = 1.0,
        quality: str = "hires",
        spot_diameter_fullres: float = 50,
) -> AnnData:
    """\
    Read Old Spatial Transcriptomics data
    Parameters
    ----------
    count_matrix_file
        Path to count matrix file.
    spatial_file
        Path to spatial location file.
    image_file
        Path to the tissue image file
    library_id
        Identifier for the visium library. Can be modified when concatenating multiple adata objects.
    scale
        Set scale factor.
    quality
        Set quality that convert to stlearn to use. Store in anndata.obs['imagecol' & 'imagerow']
    spot_diameter_fullres
        Diameter of spot in full resolution
    Returns
    -------
    AnnData
    """

    adata = scanpy.read_text(count_matrix_file)
#     spot_df = pd.read_csv(spatial_file, index_col=0)
    spot_df = pd.read_csv(spatial_file, sep="\t")
    spot_df.index = spot_df["x"].astype(str) + "x" + spot_df["y"].astype(str)
#     comm_index = pd.Series(list(set(spot_df.index).intersection(set(adata.obs_names))))
    comm_index = adata.obs_names.intersection(spot_df.index)
#     print(comm_index)
    adata = adata[comm_index]
#     adata.obs["imagecol"] = spot_df["X"]
#     adata.obs["imagerow"] = spot_df["Y"]
    adata.obs["imagecol"] = spot_df["pixel_x"]
    adata.obs["imagerow"] = spot_df["pixel_y"]
    stlearn.add.image(
        adata,
        library_id=library_id,
        quality=quality,
        imgpath=image_file,
        scale=scale,
        spot_diameter_fullres=spot_diameter_fullres,
    )

    return adata


In [25]:
BASE_PATH=Path("/clusterdata/uqxtan9/Xiao/STimage/"
                "dataset/breast_cancer_oldST_STNet")
TILING_PATH= BASE_PATH / "tile"
TILING_PATH.mkdir(parents=True, exist_ok=True)

In [24]:
dataset = pd.read_csv(BASE_PATH / "dataset.csv", index_col=0)

In [25]:
dataset

,sample,count_matrix,spot_coordinates,histology_image
0,BC23287_C1,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...
1,BC23287_C2,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...
2,BC23287_D1,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...
3,BC23450_D2,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...
4,BC23450_E1,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...
...,...,...,...,...
63,BC23377_C2,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...
64,BC23377_D1,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...
65,BC23803_D2,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...
66,BC23803_E1,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...,/clusterdata/uqxtan9/Xiao/STimage/dataset/brea...


In [26]:
stain_normalization = True
template_sample = "BC23287_C1"
adata_list = []
normaliser = None
for index, row in dataset.iterrows():
        img_path = row["histology_image"]
        Sample = row["sample"]
        cm_path = row["count_matrix"]
        spot_path = row["spot_coordinates"]

        adata = ReadOldST(count_matrix_file=cm_path,
                                  spatial_file=spot_path,
                                  image_file=img_path,
                                  library_id=Sample,
                                  quality="fulres")
        if stain_normalization and Sample == template_sample:
            print("fitting template image for stain normalizer")
            template_img = adata.uns["spatial"][Sample]['images']["fulres"]
            template_img = Image.fromarray(template_img.astype("uint8"))

            normaliser = IterativeNormaliser(normalisation_method='vahadane', standardise_luminosity=True)
            normaliser.fit_target(scale_img(template_img))
        
        st.pp.log1p(adata)
        adata_list.append(adata)

Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
fitting template image for stain normalizer
Log transformation step is finished in adata.X


Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Trying to set attribute `.obs` of view, copying.


...

Added tissue image to the object!
Log transformation step is finished in adata.X


Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


In [37]:
tile_filtering_threshold = 0.8
tile_size=299
for i, adata in enumerate(adata_list):
    if tile_filtering_threshold < 1:
        print("filtering out tiles where tissue area less than {} of total tile area".format(
                    tile_filtering_threshold))
        calculate_bg(adata, crop_size=tile_size, stain_normaliser=normaliser)
        tile_to_remove = sum(adata.obs["tissue_area"] < tile_filtering_threshold)
        adata = adata[adata.obs["tissue_area"] >= tile_filtering_threshold].copy()
        print("{} tiles with low tissue coverage are removed".format(tile_to_remove))
        adata_list[i] = adata

    tiling(adata, out_path=TILING_PATH, crop_size=tile_size, stain_normaliser=normaliser)

filtering out tiles where tissue area less than 0.8 of total tile area


Tiling image: 100%|██████████ [ time left: 00:00 ]


0 tiles with low tissue coverage are removed


Tiling image: 100%|██████████ [ time left: 00:00 ]


filtering out tiles where tissue area less than 0.8 of total tile area


...

Tiling image: 100%|██████████ [ time left: 00:00 ]


filtering out tiles where tissue area less than 0.8 of total tile area


Tiling image: 100%|██████████ [ time left: 00:00 ]


34 tiles with low tissue coverage are removed


Tiling image: 100%|██████████ [ time left: 00:00 ]


In [40]:
adata_all = adata_list[0].concatenate(
            adata_list[1:],
            batch_key="library_id",
            uns_merge="unique",
            batch_categories=[list(d.keys())[0]
                              for d in [adata_list[i].uns["spatial"]
                                        for i in range(len(adata_list))
                                        ]
                              ],
        )

In [41]:
adata_all.write(BASE_PATH / "all_adata.h5ad")

In [26]:
BASE_PATH=Path("/clusterdata/uqxtan9/Q1851/Xiao/Working_project/her2st")
TILING_PATH= BASE_PATH / "tile"
TILING_PATH.mkdir(parents=True, exist_ok=True)
DATA_PATH = Path("/clusterdata/uqxtan9/Q1851/Xiao/Working_project/STimage/development/data/her2st/data")

In [27]:
# BASE_PATH=Path("/home/uqxtan9/Q1851/Xiao/Working_project/her2st")
# TILING_PATH= BASE_PATH / "tile"
# TILING_PATH.mkdir(parents=True, exist_ok=True)
# DATA_PATH = Path("/home/uqxtan9/Q1851/Xiao/Working_project/STimage/development/data/her2st/data")

In [28]:
# prefix= "/clusterdata/uqxtan9/Q1851/Xiao/Working_project/STimage/development/data/her2st/data"

In [29]:
dataset = pd.DataFrame()

In [30]:
samples = [i.stem for i in DATA_PATH.glob("./ST-cnts/*.tsv")]

In [31]:
dataset["sample"] = samples

In [32]:
def get_img(img_dir, name):
    pre = img_dir+'/'+name[0]+'/'+name
    fig_name = os.listdir(pre)[0]
    path = pre+'/'+fig_name
    return path

In [33]:
dataset["count_matrix"] = [str(DATA_PATH) + "/ST-cnts/{}.tsv".format(i) for i in samples]
dataset["spot_coordinates"] = [str(DATA_PATH) + "/ST-spotfiles/{}_selection.tsv".format(i) for i in samples]
dataset["histology_image"] = [get_img((str(DATA_PATH)+"/ST-imgs"), i) for i in samples]


In [34]:
# dataset = dataset.replace(str(DATA_PATH), prefix)

In [35]:
stain_normalization = True
template_sample = "C1"
adata_list = []
normaliser = None
for index, row in dataset.iterrows():
        img_path = row["histology_image"]
        Sample = row["sample"]
        cm_path = row["count_matrix"]
        spot_path = row["spot_coordinates"]

        adata = ReadOldST(count_matrix_file=cm_path,
                                  spatial_file=spot_path,
                                  image_file=img_path,
                                  library_id=Sample,
                                  quality="fulres")
        if stain_normalization and Sample == template_sample:
            print("fitting template image for stain normalizer")
            template_img = adata.uns["spatial"][Sample]['images']["fulres"]
            template_img = Image.fromarray(template_img.astype("uint8"))

            normaliser = IterativeNormaliser(normalisation_method='vahadane', standardise_luminosity=True)
            normaliser.fit_target(scale_img(template_img))
        
        st.pp.log1p(adata)
        adata_list.append(adata)

Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Trying to set attribute `.obs` of view, copying.


...

Added tissue image to the object!
Log transformation step is finished in adata.X


Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


In [36]:
TILING_PATH= BASE_PATH / "tile_no_normal_224"
TILING_PATH.mkdir(parents=True, exist_ok=True)
tile_size=224
for i, adata in enumerate(adata_list):
    tiling(adata, out_path=TILING_PATH, crop_size=tile_size, target_size=tile_size)

Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [ time left: 00:00 ]
Tiling image: 100%|██████████ [

In [37]:
adata_all = adata_list[0].concatenate(
            adata_list[1:],
            batch_key="library_id",
            uns_merge="unique",
            batch_categories=[list(d.keys())[0]
                              for d in [adata_list[i].uns["spatial"]
                                        for i in range(len(adata_list))
                                        ]
                              ],
        )

In [38]:
adata_all.write(BASE_PATH / "all_adata_no_normal_224.h5ad")

In [20]:
tile_filtering_threshold = 0.8
tile_size=299
TILING_PATH= BASE_PATH / "tile"
TILING_PATH.mkdir(parents=True, exist_ok=True)

for i, adata in enumerate(adata_list):
    if tile_filtering_threshold < 1:
        print("filtering out tiles where tissue area less than {} of total tile area".format(
                    tile_filtering_threshold))
        calculate_bg(adata, crop_size=tile_size, stain_normaliser=normaliser)
        tile_to_remove = sum(adata.obs["tissue_area"] < tile_filtering_threshold)
        adata = adata[adata.obs["tissue_area"] >= tile_filtering_threshold].copy()
        print("{} tiles with low tissue coverage are removed".format(tile_to_remove))
        adata_list[i] = adata

    tiling(adata, out_path=TILING_PATH, crop_size=tile_size, stain_normaliser=normaliser)

filtering out tiles where tissue area less than 0.8 of total tile area


Tiling image: 100%|██████████ [ time left: 00:00 ]


11 tiles with low tissue coverage are removed


Tiling image: 100%|██████████ [ time left: 00:00 ]


filtering out tiles where tissue area less than 0.8 of total tile area


...

Tiling image: 100%|██████████ [ time left: 00:00 ]


filtering out tiles where tissue area less than 0.8 of total tile area


Tiling image: 100%|██████████ [ time left: 00:00 ]


10 tiles with low tissue coverage are removed


Tiling image: 100%|██████████ [ time left: 00:00 ]


In [21]:
adata_all = adata_list[0].concatenate(
            adata_list[1:],
            batch_key="library_id",
            uns_merge="unique",
            batch_categories=[list(d.keys())[0]
                              for d in [adata_list[i].uns["spatial"]
                                        for i in range(len(adata_list))
                                        ]
                              ],
        )

In [22]:
adata_all.write(BASE_PATH / "all_adata.h5ad")